In [1]:
import unittest
import numpy as np
import operator
from functools import reduce

In [2]:
input_str = open('input.txt').read().strip()
input_lengths = list(map(int, input_str.split(',')))

In [3]:
def mklengths(s):
    return list(map(ord, s)) + [17, 31, 73, 47, 23]

In [4]:
input_lengths_part2 = mklengths(input_str)

In [5]:
class Ring(object):
    def __init__(self, size=256):
        self.l = np.arange(size)
        self.len = size
        self.cur = 0
        self.skip = 0

    def checksum(self):
        return self.l[0] * self.l[1]
    
    def knot(self, length):
        # bring the current to the hed of the list
        tmp = np.roll(self.l, -self.cur)
        #print(f'1> {tmp}')
        # reverse the elements
        tmp[0:length] = tmp[0:length][::-1]
        #print(f'2> {tmp}')
        # roll it back to the original positions
        self.l = np.roll(tmp, self.cur)
        # now make the current and skip modifications
        self.cur = (self.cur + length + self.skip) % self.len
        self.skip += 1
        return self
    
    def knot_all(self, lengths):
        for l in lengths:
            self.knot(l)
        return self
    
    def do_rounds(self, lengths, rounds=64):
        for i in range(rounds):
            self.knot_all(lengths)
        return self
    
    def dense_hash(self):
        tmp = self.l
        dense = []
        for i in range(16):
            chunk = tmp[0:16]
            tmp = tmp[16:]
            dense.append(reduce(operator.xor, chunk))
        #return ''.join(lambda x: f'{x:2x}' for x in dense)
        return ''.join([f'{x:02x}' for x in dense])
    
    def __str__(self):
        outl = []
        for i, c in enumerate(self.l):
            outl.append(f'[{c}]' if i == self.cur else f'{c}')
        return ' '.join(outl)
    
    __repr__ = __str__

In [6]:
class TestIt(unittest.TestCase):
    def test01(self):
        r = Ring(5)
        r.knot(3)
        self.assertEqual(str(r), '2 1 0 [3] 4')
    def test02(self):
        r = Ring(5)
        r.knot_all([3, 4])
        self.assertEqual(str(r), '4 3 0 [1] 2')
    def test03(self):
        r = Ring(5)
        r.knot_all([3, 4, 1])
        self.assertEqual(str(r), '4 [3] 0 1 2')
    def test04(self):
        r = Ring(5)
        r.knot_all([3, 4, 1, 5])
        self.assertEqual(str(r), '3 4 2 1 [0]')
    def test05(self):
        r = Ring(5)
        r.knot_all([3, 4, 1, 5])
        self.assertEqual(r.checksum(), 12)
    def test06(self):
        r = Ring(256)
        lengths = mklengths('')
        r.do_rounds(lengths)
        self.assertEqual(r.dense_hash(), 'a2582a3a0e66e6e86e3812dcb672a272')
    def test07(self):
        r = Ring(256)
        lengths = mklengths('AoC 2017')
        r.do_rounds(lengths)
        self.assertEqual(r.dense_hash(), '33efeb34ea91902bb2f59c9920caa6cd')
    def test08(self):
        r = Ring(256)
        lengths = mklengths('1,2,3')
        r.do_rounds(lengths)
        self.assertEqual(r.dense_hash(), '3efbe78a8d82f29979031a4aa0b16a9d')
    def test09(self):
        r = Ring(256)
        lengths = mklengths('1,2,4')
        r.do_rounds(lengths)
        self.assertEqual(r.dense_hash(), '63960835bcdc130f0b66d7ff4f6a5a8e')


In [7]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestIt)
unittest.TextTestRunner(verbosity=2).run(suite)

test01 (__main__.TestIt) ... ok
test02 (__main__.TestIt) ... ok
test03 (__main__.TestIt) ... ok
test04 (__main__.TestIt) ... ok
test05 (__main__.TestIt) ... ok
test06 (__main__.TestIt) ... ok
test07 (__main__.TestIt) ... ok
test08 (__main__.TestIt) ... ok
test09 (__main__.TestIt) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.167s

OK


<unittest.runner.TextTestResult run=9 errors=0 failures=0>

In [8]:
r = Ring(256)

In [9]:
r.knot_all(input_lengths)
print(f'part 1 answer: {r.checksum()}')

part 1 answer: 38628


In [10]:
r = Ring(256)

In [11]:
r.do_rounds(input_lengths_part2)
print(f'part 2 answer: {r.dense_hash()}')

part 2 answer: e1462100a34221a7f0906da15c1c979a
